In [1]:
import string
import random
import string
import pickle
from string import ascii_letters
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import sqlite3
from datetime import datetime,timedelta
import pandas as pd
import re
from chatbot_respuesta import *

In [2]:
#menú de comando
def opciones(intencion_previa, entradaUsuario):
    if intencion_previa=='saludos' and entradaUsuario=='1':
        print('''Escribe una de las siguiente opciones:
                 *Servicio 
                 *Ubicación
                 *Agendar cita
                 *Cancelar cita
                 *Modificar cita
                 *Pagos
                 *Cotización de presupuestos
                 *Garantía
                 *Reclamo
                 ''')

CREAR TOKEN

In [3]:
#scopes=['https://www.googleapis.com/auth/calendar']
#flow=InstalledAppFlow.from_client_secrets_file('CREDENTIAL.json',scopes=scopes)

In [4]:
credentials=pickle.load(open('token.pkl','rb'))
service=build('calendar','v3',credentials=credentials)

In [5]:
##Crear bases de datos

import sqlite3


conn = sqlite3.connect('Calendario_citas.db')

cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Citas'")
if cursor.fetchone() is None:
    
    cursor.execute('''CREATE TABLE Citas
                    (id_cita TEXT NOT NULL,
                    calendar_id TEXT NOT NULL,
                    servicio TEXT NOT NULL,
                    nombre TEXT NOT NULL,
                    description TEXT NOT NULL,
                    hora_de_inicio INTEGER,
                    hora_finalizacion INTEGER,
                    correo TEXT NOT NULL);''')

    print("Bases de datos creado")
else:
    print("Table already exists")
conn.close()

Table already exists


PROCESO DE AGENDAR CITAS

In [6]:
def range_alpha(start_letter, end_letter):
    return ascii_letters[
    ascii_letters.index(start_letter):ascii_letters.index(end_letter) + 1
  ]

def codigo_cita():
    letras=range_alpha('a', 'v')
    characters = letras + string.digits 
    cita_id = ''.join(random.choice(characters) for i in range(8))
    return cita_id+'5'


def mostrarDatos(dictinfo):
    msgconf = 'Para terminar el proceso de agendar su cita, confirme que los datos ingresados son correctos.'
    print(msgconf,'\n')
    for key in dictinfo.keys():    
        print(key, ':', dictinfo[key])

def verificar_correo():
   correo_verificar=''
   correo=''
   while correo_verificar!=True:
    entradaUsuario=input()
    reCorreo= r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|\"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*\")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"
    correo_verificar= re.match(reCorreo,entradaUsuario) is not None 
    if correo_verificar==True:
        correo=entradaUsuario
        return correo
    else:
        print('El correo ingresado no es válido, ingrese nuevamente')

        

In [7]:
def getInfo(msgInicial, listaConsultas, servicio):
    print(msgInicial)
    for i, pregChat in enumerate(listaConsultas):
        print(pregChat)
        if i ==0:
            entradaUsuario = input()
            nombre = entradaUsuario
            
        if i ==1:
            entradaUsuario = input()
            fecha = str(entradaUsuario+' ')
            
        if i ==2:
            entradaUsuario = input()
            hora = str(entradaUsuario+':00')
            fecha= fecha+hora
            start_time= datetime.strptime(fecha, "%d/%m/%Y %H:%M:%S")
            end_time=start_time+timedelta(hours=1)

        if i ==3:
            correo= verificar_correo() 
        if i ==4:
            entradaUsuario = input()
            description = entradaUsuario 

    return {'nombre':nombre, 'descripcion':description, 'hora_de_inicio':start_time, 'hora_finalización':end_time,'correo':correo}
            
msgInicial = "Para consultar disponibilidad indiquenos los siguientes datos:"
listaConsultas = ['por favor indiqueme el nombre','Ingrese fecha en formato *DD/MM/AAAA* ','Ingrese hora en formato de 24 horas *hora:min*','Ingrese su correo electrónico','Porqué está agendando esta cita?']


In [8]:
def crear_evento(nombre,servicio,id_cita,description,start_time,end_time,correo,calendar_id,locacion):
    event = { 
        'summary': (nombre + '  servicio: ' + servicio),
        'id':id_cita,
          'location': locacion,
          'description': description,
          'start': {
            'dateTime': start_time.strftime('%Y-%m-%dT%H:%M:%S'),
            'timeZone': 'America/Panama',
          },
          'end': {
            'dateTime': end_time.strftime('%Y-%m-%dT%H:%M:%S'),
            'timeZone': 'America/Panama',
          },
          'attendees': [
            {'email': correo}
          ],
          'reminders': {
            'useDefault': False,
            'overrides': [
              {'method': 'email', 'minutes': 24 * 60},
              {'method': 'popup', 'minutes': 10},
            ],
          },
        }
    service.events().insert(calendarId=calendar_id, body=event).execute()

In [9]:
def aggdatos_sql(datos):
    conn = sqlite3.connect('Calendario_citas.db')
    cursor = conn.cursor()
    ingresar= cursor.execute("INSERT INTO Citas VALUES (:id_cita, :calendar_id,:modalidad,:servicio, :nombre,:descripcion,:hora_de_inicio,:hora_finalización,:correo)", datos) 
    conn.commit()
    conn.close()
    return ingresar

In [10]:
import qrcode 
from PIL import Image, ImageDraw

def generar_qr(id_cita,infoCita):
    data = infoCita
    img = qrcode.make(data)
    qr_name=str(id_cita)+'.png'
    qr=img.save(qr_name,'C:\Users\Dell\Proyecto Hackathon\codigos_qr')
    return qr


In [11]:
def confirmar_datos_virtual(intencion,infoCita,servicio,calendar_id):
    if intencion=='Afirmación' :
            id_citas= codigo_cita()
            infoCita['id_cita']=id_citas
            infoCita['calendar_id']=calendar_id
            crear_evento(infoCita['nombre'],servicio,id_citas,infoCita['descripcion'],infoCita['hora_de_inicio'],infoCita['hora_finalización'],infoCita['correo'],infoCita['calendar_id'],infoCita['modalidad'])
            aggdatos_sql(infoCita)
            print('Su cita ha sido agendada con éxito, su id de cita es:',id_citas, 'en que más puedo ayudarte?',)
    elif intencion=='Negación':
            print('Ok, cita no agendada. En que más puedo ayudarte?')

def confirmar_datos_presencial(intencion,infoCita,servicio,calendar_id):
    if intencion=='Afirmación' :
            id_citas= codigo_cita()
            infoCita['id_cita']=id_citas
            infoCita['calendar_id']=calendar_id
            crear_evento(infoCita['nombre'],servicio,id_citas,infoCita['descripcion'],infoCita['hora_de_inicio'],infoCita['hora_finalización'],infoCita['correo'],infoCita['calendar_id'],infoCita['modalidad'])
            aggdatos_sql(infoCita)
            generar_qr(id_citas,infoCita)
            print('Su cita ha sido agendada con éxito, su id de cita es:',id_citas, ' así como su código qr que debe presentar al momento de llegar a nuestras oficinas. En que más puedo ayudarte?',)
    elif intencion=='Negación':
            print('Ok, cita no agendada. En que más puedo ayudarte?')

In [12]:
id_asesoria='longstoryshort51@gmail.com'
id_atencióncliente='1e8f7a6f98f1ef21fe6c91c81f4bf84ea6715d776b3621517fb74a04fec2badd@group.calendar.google.com'


#Los id de calendario depende de la cuenta a ingresar para obtener el token.

In [13]:
def citas(intencion_previa,entradaUsuario,intencion):
    
    if intencion_previa=='Agendar Cita' and entradaUsuario == '1':
        msg1='''Para agendar una cita virtual indique el servicio que desea:
                 
                                1. Asesoría de compras
                                2. Atención al cliente'''
        print(msg1)
        entradaUsuario=input()
        modalidad='virtual'
        if  entradaUsuario == '1':
            servicio='Asesoría'
            calendar_id=id_asesoria
            infoCita = getInfo(msgInicial, listaConsultas, servicio)
            infoCita['servicio']=servicio
            infoCita['modalidad']=modalidad
            mostrarDatos(infoCita)
            entradaUsuario = input()
            intencion = chatbot_respuesta(entradaUsuario)
            confirmar_datos_virtual(intencion,infoCita,servicio,calendar_id)
            


        elif entradaUsuario == '2':
            servicio='Atención al cliente'
            calendar_id=id_atencióncliente
            infoCita = getInfo(msgInicial, listaConsultas, servicio)
            infoCita['servicio']=servicio
            infoCita['modalidad']=modalidad
            mostrarDatos(infoCita)
            entradaUsuario = input()
            respuesta,intencion = chatbot_respuesta(entradaUsuario)
            confirmar_datos_virtual(intencion,infoCita,servicio,calendar_id)
            
        else: 
            print('Opción no válida')
            
    if intencion_previa=='Agendar Cita' and entradaUsuario == '2':
        msg1='''Para agendar una cita virtual indique el servicio que desea:
                 
                                1. Asesoría de compras
                                2. Atención al cliente'''
        modalidad='Presencial'                         
        print(msg1)
        entradaUsuario=input()

        if  entradaUsuario == '1':
            locacion='Ciudad de Panamá'
            servicio='Asesoría'
            calendar_id=id_asesoria
            infoCita = getInfo(msgInicial, listaConsultas, servicio)
            infoCita['servicio']=servicio
            infoCita['modalidad']=modalidad
            mostrarDatos(infoCita)
            entradaUsuario = input()
            respuesta,intencion = chatbot_respuesta(entradaUsuario)
            confirmar_datos_presencial(intencion,infoCita,servicio,calendar_id)
            


        elif entradaUsuario == '2':
            locacion='Ciudad de Panamá'
            servicio='Atención al cliente'
            calendar_id=id_atencióncliente
            infoCita = getInfo(msgInicial, listaConsultas, servicio)
            infoCita['servicio']=servicio
            infoCita['modalidad']=modalidad
            mostrarDatos(infoCita)
            entradaUsuario = input()
            respuesta, intencion = chatbot_respuesta(entradaUsuario)
            confirmar_datos_presencial(intencion,infoCita,servicio,calendar_id)
            

         
        else: 
            print('Opción no válida')
        
 

In [14]:
citas('Agendar Cita','2','Agendar Cita')

Para agendar una cita virtual indique el servicio que desea:
                 
                                1. Asesoría de compras
                                2. Atención al cliente
Para consultar disponibilidad indiquenos los siguientes datos:
por favor indiqueme el nombre
Ingrese fecha en formato *DD/MM/AAAA* 
Ingrese hora en formato de 24 horas *hora:min*



KeyboardInterrupt



In [ ]:
def modificar_fecha():
  listaConsultas = ['Ingrese fecha en formato *DD/MM/AAAA* ','Ingrese hora en formato de 24 horas *hora:min*']
  entradaUsuario=input()
  for i, pregChat in enumerate(listaConsultas):
          print(pregChat)
          if i ==1:
              fecha = str(entradaUsuario+' ')
              
          if i ==2:
              hora = str(entradaUsuario+':00')
              fecha= fecha+hora
              start_time= datetime.strptime(fecha, "%d/%m/%Y %H:%M:%S")
              end_time=start_time+timedelta(hours=1)
              
              return start_time,end_time

                

In [ ]:
def verificar_id(id_citas):
    conn = sqlite3.connect('Calendario_citas.db')
    c = conn.cursor()
    c.execute("SELECT COUNT(*) FROM Citas WHERE id_cita = ?", (id_citas,))
    existe = c.fetchone()[0]
    conn.close()
    buscar_id=''
    while buscar_id != True:
        if existe > 0:
            buscar_id=True
            return buscar_id
        else:
            buscar_id=False
            msg="No se ha encontrado alguna cita registrada con el ID proporcionado, por favor ingreselo nuevamente o ingrese *exit*"
            print(msg)

            
def traer_datos(id_citas):
    conn = sqlite3.connect('Calendario_citas.db')
    c = conn.cursor()
    c.execute("SELECT * FROM Citas WHERE id_cita = ?", (id_citas,))
    fila = c.fetchone()

    conn.close()
    if fila:
        id_citas=fila[0]
        calendar_id= fila[1]
        modalidad=fila[2]
        servicio=fila[3]
        decripcion=fila[4]
        start_time=fila[5]
        end_time=fila[6]
        correo=fila[7]
        
        return {'id_citas':id_citas,'calendar_id':calendar_id,'modalidad':modalidad, 'descripcion':decripcion, 'hora_de_inicio':start_time, 'hora_finalización':end_time,'correo':correo}
    else:
        print("El id no existe en la tabla")


In [ ]:
def cambiar_info(entradaUsuario, id_citas):
    infoCita=traer_datos(id_citas)
    evento = service.events().get(calendarId=infoCita['calendar_id'], eventId=id_citas).execute()
    if entradaUsuario=='1':
        print('''Seleccione la opción que necesita:
                 1. Asesoría
                 2. Atención al cliente ''')
        entradaUsuario=input()
        if entradaUsuario=='1':
            calendar_id=id_asesoria
            evento = service.events().get(calendarId=calendar_id, eventId=id_citas).execute()
            evento['summary'] = 'Asesoría'
        elif entradaUsuario=='1':
            calendar_id=id_atencióncliente
            evento = service.events().get(calendarId=calendar_id, eventId=id_citas).execute()
            evento['summary'] = 'Atención al cliente'

        
    elif entradaUsuario=='2':
        print('''Seleccione la modalidad que desea:
                 1. Virtual
                 2. Presencial ''')

        entradaUsuario=input()
        if entradaUsuario=='1':
          evento['location'] = 'Virtual'

        elif entradaUsuario=='2':
          evento['location'] = 'Presencial'
          generar_qr(id_citas,infoCita)
          msg='Su modalidad ha sido cambiada con éxito, se le adjunta el código QR necesario al momento de acudir a nuestras oficina'
          print(msg)
        

    elif entradaUsuario=='3':
        modificar_fecha()        
        evento['start_time'] = start_time.strftime('%Y-%m-%dT%H:%M:%S')
        evento['end_time']= end_time.strftime('%Y-%m-%dT%H:%M:%S')
        
    elif entradaUsuario=='4':
        print('Ingrese su Nombre')
        entradaUsuario=input()
        evento['nombre'] = entradaUsuario
        
    elif entradaUsuario=='5':
        print('Ingrese su correo')
        verificar_correo()
        evento['correo'] = entradaUsuario

    else:
        print('Opción no válida, ingrese nuevamente')



In [ ]:
def modificar_cita(id_citas,intencion_previa):
    buscar_id=verificar_id(id_citas)
    evento=''
    if intencion_previa=='Modificar Cita' and buscar_id== True:
      menu=''''Indique el parametro a modificar (Número):
                  1. Servicio
                  2. Modalidad
                  3. Fecha y hora
                  4. Nombre
                  5. Correo'''
      print(menu)
      entradaUsuario=input()
      cambiar_info(entradaUsuario,id_citas)
    return service.events().update(calendarId='primary', eventId=id_citas, body=evento).execute()


In [ ]:
modificar_cita('i5d2nqsv5','Modificar Cita')

In [ ]:
def eliminar_datos(id_citas):
    conn = sqlite3.connect('Calendario_citas.db')
    c = conn.cursor()
    c.execute("DELETE FROM Citas WHERE id_cita = ?",(id_citas,))
    conn.commit()

    conn.close()

def cancelar_cita(id_citas,intencion_previa):
      infoCita=traer_datos(id_citas)
      buscar_id=verificar_id(id_citas)
      if intencion_previa=='Cancelar Cita' and buscar_id== True:
         eliminar_datos(id_citas)
         service.events().delete(calendarId=infoCita['calendar_id'], eventId=id_citas).execute()


In [ ]:
entradaUsuario = ''
intencion_previa = ''
msgconf = 'Para terminar el proceso de agendar su cita, confirme que los datos ingresados son correctos.'

while entradaUsuario != 'exit':
    entradaUsuario = input()
    respuesta, intencion = chatbot_respuesta(entradaUsuario)
    opciones(intencion_previa, entradaUsuario)
    citas(intencion_previa,entradaUsuario,intencion)
    cancelar_cita(id_citas,intencion_previa)
    cambiar_info(entradaUsuario, id_citas)
    intencion_previa = intencion  
    print(respuesta)
    

In [ ]:
import sqlite3

def cotizacion(id1):
    conn = sqlite3.connect('paquetes.db')
    c = conn.cursor()
    c.execute("SELECT precio FROM paquetes WHERE id in (?)", (id1,))
    precio = c.fetchone()[0]
    conn.close()
    return precio

# Ejemplo de uso
cotizacion1 = cotizacion(4)
print("La cotizacion es: ",cotizacion1)


def verificar_id_paquetes(id_paq):
    conn = sqlite3.connect('paquetes.db')
    c = conn.cursor()
    c.execute("SELECT COUNT(*) FROM paquetes WHERE id = ?", (id_paq,))
    existe = c.fetchone()[0]
    conn.close()
    id_encontrado=''
    while id_encontrado != True:
        if existe > 0:
            id_encontrado==True
            id_paq=id_paq
            return id_encontrado,id_paq
        else:
            msg='Id no válido, ingrese nuevamente'
            print(msg)
            entradaUsuario=input()
def cotizar_presupuesto():
    intencion_previa=input()
    if intencion_previa=='Cotización de presupuesto':
        msg='Ingrese el id de los items que desea (uno por uno)'
        print(msg)
        id=''
        sum_cotizacion=0
        while id!='listo':
            entradaUsuario=input()
            id=entradaUsuario
            sum_cotizacion=sum_cotizacion+cotizacion(id)
            msg2='Si ya no desea continuar ingrese *listo*'
            print(msg2)
        return sum_cotizacion